### AI Agents and Big Data Analytics - Better Together
Up to 80% of the data that enterprises generate is unstructured and semi-structured data. Your AI Agents will provide richer insights by querying TBs of data stored in S3 data lake.<br><br>
Do AI developers need to learn data engineering skills such as distributed spark processing, metadata catalogs, complex distributed SQL queries, NoSQL queries etc to query TBs and PBs of data in data lakes?<br><br>
Fortunately, the answer is no. You just need to know the basics. You can use Strands Agents, built-in strands tools such as use_aws, and MCP server such as AWS Data Processing MCP Server to accomplish what most data engineers can, while writing few lines of code.<br><br>
This notebook will help you bridge the gap between a data engineer who knows how to use AWS Glue, Amazon Athena with an AI engineer who knows how to use Strands SDK, Amazon Bedrock. You will be able to do sophisticated analytics with natural language questions.<br>


### GOAL - Perform big data analytics using AI Agents without writing big data analytics code. <br>
The AI Agent will use built-in strands tools and MCP server to get the job done. 

### In this Notebook
You will learn how to use AI Agents to discover metadata of big-data stored in parquet files using Agent tools and MCP server that in turn uses AWS Glue and Amazon Athena.<br><br>
We will use NYC Taxi and Limousine Commission (TLC) data

### Environment Setup
Install required dependencies for the notebook including Strands SDK, AWS SDK, and MCP client libraries.

In [1]:
# Install all required packages from requirements.txt
!pip install -r requirements.txt --quiet --upgrade

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
autogluon-multimodal 1.3.0 requires nvidia-ml-py3<8.0,>=7.352.0, which is not installed.
dash 2.18.1 requires dash-core-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-html-components==2.0.0, which is not installed.
dash 2.18.1 requires dash-table==5.0.0, which is not installed.
jupyter-ai 2.31.4 requires faiss-cpu!=1.8.0.post0,<2.0.0,>=1.8.0, which is not installed.
aiobotocore 2.21.1 requires botocore<1.37.2,>=1.37.0, but you have botocore 1.40.53 which is incompatible.
autogluon-common 1.3.0 requires pandas<2.3.0,>=2.0.0, but you have pandas 2.3.3 which is incompatible.
autogluon-core 1.3.0 requires pandas<2.3.0,>=2.0.0, but you have pandas 2.3.3 which is incompatible.
autogluon-features 1.3.0 requires pandas<2.3.0,>=2.0.0, but you have pandas 2.3.3 which is incompatible.
autogluon-multimod

In [2]:
# Lab configurations

# CloudFormation Stack Name - change it as needed
STACK_NAME = "AwsLabBigDataAgentStack"

# Region
import boto3
region = boto3.Session().region_name

# Setup the logging
import logging

logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    datefmt='%m/%d %H:%M:%S',
    filename='strands_debug.log'
)


## Data Pipeline using Strands Agent
### Overview

This agent downloads data from a URL and uploads it to an S3 bucket using dynamic partitioning.
### Capabilities

    Downloads data from specified URLs
    Validates downloaded content
    Uploads data to S3 with dynamic partitioning
    Handles error scenarios gracefully
    Provides logging and status updates

### Prerequisites

    AWS credentials configured
    Required Python packages:
        strands
        boto3

### Use Cloudformation template to create S3 bucket

In [5]:
%%time
# This cell will deploy AWS resources including S3 bucket and IAM roles. It requires about 3 mins to complete.
from utils.deploy_cfn import deploy_infrastructure
deploy_infrastructure(STACK_NAME)

🚀 Deploying stack: AwsLabBigDataAgentStack
🏷️  Suffix: workshop
🆕 Creating new stack: AwsLabBigDataAgentStack
✅ Stack creation initiated. Stack ID: arn:aws:cloudformation:us-west-2:000384376621:stack/AwsLabBigDataAgentStack/1d4e2540-aa33-11f0-ac1c-06b8fd3a75d7
⏳ Waiting for stack creation to complete...
🎉 Stack created successfully!
CPU times: user 121 ms, sys: 27.3 ms, total: 148 ms
Wall time: 3min


[{'OutputKey': 'BucketArn',
  'OutputValue': 'arn:aws:s3:::000384376621-us-west-2-big-data-agent-workshop',
  'Description': 'ARN of the created S3 bucket',
  'ExportName': 'AwsLabBigDataAgentStack-BucketArn'},
 {'OutputKey': 'RoleName',
  'OutputValue': 'AwsLabBigDataAgentStack-BigDataAgentRole-2oCLXxnlPxXX',
  'Description': 'Name of the created IAM role',
  'ExportName': 'AwsLabBigDataAgentStack-RoleName'},
 {'OutputKey': 'BucketName',
  'OutputValue': '000384376621-us-west-2-big-data-agent-workshop',
  'Description': 'Name of the created S3 bucket',
  'ExportName': 'AwsLabBigDataAgentStack-BucketName'},
 {'OutputKey': 'InstanceProfileArn',
  'OutputValue': 'arn:aws:iam::000384376621:instance-profile/big-data-agent-instance-profile-workshop',
  'Description': 'ARN of the instance profile',
  'ExportName': 'AwsLabBigDataAgentStack-InstanceProfileArn'},
 {'OutputKey': 'RoleArn',
  'OutputValue': 'arn:aws:iam::000384376621:role/AwsLabBigDataAgentStack-BigDataAgentRole-2oCLXxnlPxXX',
  

### Import Dependencies and AWS Configuration
Import required libraries and configure AWS settings for the data processing workflow.

In [6]:
# Import required libraries
import os, time, boto3, json
from strands import Agent, tool
from strands.models import BedrockModel
from strands_tools import use_aws, file_write, file_read, file_write, sleep, python_repl
from datetime import datetime
from pprint import pprint
from pydantic import BaseModel, Field
from typing import Optional, List

# Bypass tool consent for automated execution
os.environ["BYPASS_TOOL_CONSENT"] = "true"
# Specify that if python_repl tool is used, it shouldnt wait for user interaction
os.environ["PYTHON_REPL_INTERACTIVE"] = "false"

model_list = ['deepseek.v3-v1:0', 'qwen.qwen3-coder-30b-a3b-v1:0', 'us.anthropic.claude-3-7-sonnet-20250219-v1:0', 'us.anthropic.claude-sonnet-4-20250514-v1:0', 'openai.gpt-oss-20b-1:0', 'openai.gpt-oss-120b-1:0']

# We will use the following model in Strands Agent
model_id = "us.anthropic.claude-3-7-sonnet-20250219-v1:0"

### Data ingestion using Strands Agents

Build a strands agents to download data from a URL and then upload that data into S3 bucket with data partitioning enabled. 

In [12]:
# Create a data ingestion agent.

data_upload_agent = Agent(
    model=BedrockModel(model_id=model_id, temperature=0.3),
    system_prompt=f"""You are an expert data engineer specializing in AWS data pipeline operations.

TASK OVERVIEW:
Download files from provided URLs and upload them to an S3 bucket with proper partitioning.

SPECIFIC REQUIREMENTS:
1. Download files from the given URLs to local storage in download folder
2. Extract S3 bucket name and other resources from stack outputs (created by CloudFormation stack: {STACK_NAME}) in {region} region
3. Verify the S3 bucket exists 
3. Upload files to the S3 bucket with appropriate partitioning structure. 
4. Use table name: data

EXECUTION GUIDELINES:
- Check S3 bucket availability before attempting uploads
- Implement proper error handling for downloads and uploads
- Use efficient partitioning strategy (e.g., by year/month/day if date fields exist)
- Verify file integrity after download
- Provide clear status updates on progress
- Handle large files appropriately to avoid memory issues
- Execute tasks sequentially without user intervention

TOOLS AVAILABLE:
- use_aws: For AWS operations
- python_repl: For data processing and file operations (ALWAYS use interactive=False)

Complete the task systematically in non-interactive mode and report final results.""",
    tools=[use_aws, python_repl]
)


# File URLs
file_urls = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-05.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2025-06.parquet", 
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-05.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-06.parquet"
]

# Direct, conversational request
data_upload_request = f"""
Hi! I need you to help me with a data upload task.

Here's what I need:
1. Get the S3 bucket name from CloudFormation stack "{STACK_NAME}"
2. Download these 4 files (only if they don't already exist locally)

{file_urls}

3. Upload them to S3 with this partition structure:
   - Extract taxi type, year, month from filename
   - Use S3 key: data/taxi_class=[type]/year=[year]/month=[month]/[filename]
"""

print(f'Starting data download')

response = data_upload_agent(data_upload_request)



Starting data download
I'll help you with this data upload task. Let's break this down into steps:

1. First, I'll get the S3 bucket name from the CloudFormation stack "AwsLabBigDataAgentStack"
2. Then download the files if they don't exist locally
3. Finally, upload them to S3 with the specified partition structure

Let's start by getting the S3 bucket name from CloudFormation:
Tool #1: use_aws


╭─── 🚀 Get CloudFormation stack outputs ────╮
│                                            │
│  Service:         cloudformation           │
│  Operation:       describe_stacks          │
│  Region:          us-west-2                │
│  Parameters:                               │
│    • StackName:   AwsLabBigDataAgentStack  │
│                                            │
╰────────────────────────────────────────────╯

Now I'll extract the S3 bucket name and verify it exists:
Tool #2: python_repl


╭───────────────────────────────────────────── Executing Python Code ─────────────────────────────────────────────╮
│ # Extract bucket name from CloudFormation outputs                                                               │
│ import json                                                                                                     │
│                                                                                                                 │
│ def extract_bucket_name(cf_response):                                                                           │
│     for stack in cf_response['Stacks']:                                                                         │
│         for output in stack['Outputs']:                                                                         │
│             if output['OutputKey'] == 'BucketName':                                                             │
│                 return output['OutputValue']                                                                    │
│     return None                                                                                                 │
│                                                                                                                 │
│ cf_response = {'Stacks': [{'StackId': 'arn:aws:cloudformation:us-west-2:000384376621:stack/AwsLabBigDataAgentSt │
│                                                                                                                 │
│ bucket_name = extract_bucket_name(cf_response)                                                                  │
│ print(f"S3 bucket name: {bucket_name}")                                                                         │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Running in standard mode...

...

Running in standard mode...

Output:

Uploading download/yellow_tripdata_2025-05.parquet to 
s3://000384376621-us-west-2-big-data-agent-workshop/data/taxi_class=yellow/year=2025/month=05/yellow_tripdata_2025-
05.parquet...
Successfully uploaded download/yellow_tripdata_2025-05.parquet
Uploading download/yellow_tripdata_2025-06.parquet to 
s3://000384376621-us-west-2-big-data-agent-workshop/data/taxi_class=yellow/year=2025/month=06/yellow_tripdata_2025-
06.parquet...
Successfully uploaded download/yellow_tripdata_2025-06.parquet
Uploading download/green_tripdata_2025-05.parquet to 
s3://000384376621-us-west-2-big-data-agent-workshop/data/taxi_class=green/year=2025/month=05/green_tripdata_2025-05
.parquet...
Successfully uploaded download/green_tripdata_2025-05.parquet
Uploading download/green_tripdata_2025-06.parquet to 
s3://000384376621-us-west-2-big-data-agent-workshop/data/taxi_class=green/year=2025/month=06/green_tripdata_2025-06
.parquet...
Successfully uploaded download/green_tripdata_2025-06.parquet

Upload Summary:
File: download/yellow_tripdata_2025-05.parquet
S3 Path: 
s3://000384376621-us-west-2-big-data-agent-workshop/data/taxi_class=yellow/year=2025/month=05/yellow_tripdata_2025-
05.parquet
Status: SUCCESS

File: download/yellow_tripdata_2025-06.parquet
S3 Path: 
s3://000384376621-us-west-2-big-data-agent-workshop/data/taxi_class=yellow/year=2025/month=06/yellow_tripdata_2025-
06.parquet
Status: SUCCESS

File: download/green_tripdata_2025-05.parquet
S3 Path: 
s3://000384376621-us-west-2-big-data-agent-workshop/data/taxi_class=green/year=2025/month=05/green_tripdata_2025-05
.parquet
Status: SUCCESS

File: download/green_tripdata_2025-06.parquet
S3 Path: 
s3://000384376621-us-west-2-big-data-agent-workshop/data/taxi_class=green/year=2025/month=06/green_tripdata_2025-06
.parquet
Status: SUCCESS

Verifying uploads by listing objects in the bucket...
Found 4 objects in s3://000384376621-us-west-2-big-data-agent-workshop/data/
- data/taxi_class=green/year=2025/month=05/green_tripdata_2025-05.parquet (1338798 bytes)
- data/taxi_class=green/year=2025/month=06/green_tripdata_2025-06.parquet (1212934 bytes)
- data/taxi_class=yellow/year=2025/month=05/yellow_tripdata_2025-05.parquet (77837865 bytes)
- data/taxi_class=yellow/year=2025/month=06/yellow_tripdata_2025-06.parquet (73542954 bytes)

✓ Code executed successfully (6.46s)
User objects in namespace: 14 items
 - bucket_name = '000384376621-us-west-2-big-data-agent-workshop'
 - download_dir = 'download'
 - url = 'https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2025-06.parquet'
 - file_path = 'download/green_tripdata_2025-06.parquet'
 - filename = 'green_tripdata_2025-06.parquet'
 - taxi_type = 'green'
 - year = '2025'
 - month = '06'
 - s3_key = 'data/taxi_class=green/year=2025/month=06/green_tripdata_2025-06.parquet'
 - s3_uri = 
's3://000384376621-us-west-2-big-data-agent-workshop/data/taxi_class=green/year=2025/month=06/green_tripdata_2025-0
6.parquet'
 - policy_str = 
'{"Version":"2012-10-17","Statement":[{"Sid":"DenyInsecureConnections","Effect":"Deny","Principal":"*","Action":"s3
:*","Resource":["arn:aws:s3:::000384376621-us-west-2-big-data-agent-workshop","arn:aws:s3:::000384376621-us-west-2-
big-data-agent-workshop/*"],"Condition":{"Bool":{"aws:SecureTransport":"false"}}},{"Sid":"DenyPublicReadWrite","Eff
ect":"Deny","Principal":"*","Action":["s3:GetObject","s3:PutObject","s3:DeleteObject","s3:GetObjectAcl","s3:PutObje
ctAcl","s3:ListBucket"],"Resource":["arn:aws:s3:::000384376621-us-west-2-big-data-agent-workshop","arn:aws:s3:::000
384376621-us-west-2-big-data-agent-workshop/*"],"Condition":{"StringNotLike":{"aws:PrincipalArn":["arn:aws:iam::000
384376621:role/AwsLabBigDataAgentStack-BigDataAgentRole-2oCLXxnlPxXX","arn:aws:iam::000384376621:root"]},"StringNot
Equals":{"aws:PrincipalServiceName":["glue.amazonaws.com","athena.amazonaws.com"]}}},{"Sid":"AllowAuthorizedAccess"
,"Effect":"Allow","Principal":{"AWS":["arn:aws:iam::000384376621:role/AwsLabBigDataAgentStack-BigDataAgentRole-2oCL
XxnlPxXX","arn:aws:iam::000384376621:root"]},"Action":["s3:GetObject","s3:PutObject","s3:DeleteObject","s3:ListBuck
et","s3:GetBucketLocation","s3:GetObjectLegalHold","s3:PutObjectLegalHold","s3:GetObjectRetention","s3:PutObjectRet
ention","s3:BypassGovernanceRetention"],"Resource":["arn:aws:s3:::000384376621-us-west-2-big-data-agent-workshop","
arn:aws:s3:::000384376621-us-west-2-big-data-agent-workshop/*"]},{"Sid":"AllowAWSServices","Effect":"Allow","Princi
pal":{"Service":["glue.amazonaws.com","athena.amazonaws.com"]},"Action":["s3:GetObject","s3:PutObject","s3:ListBuck
et","s3:GetBucketLocation"],"Resource":["arn:aws:s3:::000384376621-us-west-2-big-data-agent-workshop","arn:aws:s3::
:000384376621-us-west-2-big-data-agent-workshop/*"]}]}'
 - role = 'arn:aws:iam::000384376621:root'
 - service = 'athena.amazonaws.com'
 - local_path = 'download/green_tripdata_2025-06.parquet'

## Task Summary

I've successfully completed the data upload task. Here's a summary of what was done:

### 1. CloudFormation Stack Information
- Successfully retrieved the S3 bucket name from CloudFormation stack "AwsLabBigDataAgentStack"
- Bucket name: `000384376621-us-west-2-big-data-agent-workshop`
- Verified the bucket exists in the us-west-2 region

### 2. File Downloads
- All 4 files were already present in the local "download" directory:
  - yellow_tripdata_2025-05.parquet
  - yellow_tripdata_2025-06.parquet
  - green_tripdata_2025-05.parquet
  - green_tripdata_2025-06.parquet

### 3. S3 Upload with Partitioning
- Successfully uploaded all files to S3 with the following partition structure:
  - `data/taxi_class=[type]/year=[year]/month=[month]/[filename]`

### 4. Verification
- Verified all uploads by listing objects in the S3 bucket
- Confirmed all 4 files were uploaded with the correct partitioning structure:
  - `data/taxi_class=green/year=2025/month=05/green_tripdata_2025-05

The first step is to discover the metadata and catalog it. <br><br>
AWS Glue can help crawl the data, extract the metadata, and create a catalog using AWS Glue Catalog.<br><br>
Let's NOT write code to create AWS Glue jobs. Instead, let's write a SYSTEM PROMPT that will instruct an AI Agent to do this job for us.<br>

### Catalog Data in S3 as Glue Catalog Database and Tables
Create a Strands Agent that uses use_aws strands tool to create AWS Glue crawlers and catalog the S3 data automatically.

In [13]:
from utils_big_data import load_system_prompt_from_file
# Let's load the system prompt from file
crawl_task = load_system_prompt_from_file("crawl_task_prompt.txt", stack_name = STACK_NAME)
crawl_task

'I have files in an S3 folder and need you to create a Glue Crawler to crawl them and create/update a Glue Catalog. Use the use_aws tool to complete this task.\n\nStrictly use the specified AWS region where CFN stack AwsLabBigDataAgentStack is deployed.\n\nIMPORTANT: \n2. Use the files under data/ prefix.\n\nPlease perform these tasks using the use_aws tool:\n\n1. **Find location of the data**\n-- Check Outputs of CFN stack AwsLabBigDataAgentStack to find S3 bucket and the IAM role.\n\n2. **Check if database exists**: \n-- If the database already exits and it points to the bucket, delete all tables in that database.\n-- If the database does not exist, create it. The database name must include AwsLabBigDataAgentStack\n\n3. **Always create a glue crawler**:\n   - Do not use an existing one.\n   - Specify crawler name, role, database name, s3 target path and other configuration details.\n   - Use the glue crawler to crawl the data and create/update the glue catalog\n\n4. **Run the crawler

In [14]:
# Initialize Claude 3.7 Sonnet model via Bedrock
model = BedrockModel(model_id=model_id, temperature=0.1)  # Low temperature for consistent structured output

# Create the Strands Agent. 
# use_aws tool will create a glue crawler and create glue catalog db and table schema
# sleep tool will help the agent wait until the glue crwler job is finished.
# This task might take a few minutes. You can go to AWS Console and see the Glue Crawler jobs that are being created.
crawl_agent = Agent(model=model, tools=[use_aws, sleep])

crawl_response = crawl_agent(crawl_task)
print(crawl_response)

I'll help you create a Glue Crawler to crawl your S3 files and create/update a Glue Catalog. Let's follow the steps you outlined.

### Step 1: Find location of the data

First, I'll check the outputs of the CloudFormation stack "AwsLabBigDataAgentStack" to find the S3 bucket and IAM role.
Tool #1: use_aws


╭─── 🚀 Get CloudFormation stack outputs ────╮
│                                            │
│  Service:         cloudformation           │
│  Operation:       describe_stacks          │
│  Region:          us-east-1                │
│  Parameters:                               │
│    • StackName:   AwsLabBigDataAgentStack  │
│                                            │
╰────────────────────────────────────────────╯

I need to find the correct region where the CloudFormation stack is deployed. Let's check other regions:
Tool #2: use_aws


╭─ 🚀 Get CloudFormation stack outputs in us-west-2 ─╮
│                                                    │
│  Service:         cloudformation                   │
│  Operation:       describe_stacks                  │
│  Region:          us-west-2                        │
│  Parameters:                                       │
│    • StackName:   AwsLabBigDataAgentStack          │
│                                                    │
╰────────────────────────────────────────────────────╯

Great! I found the CloudFormation stack in the us-west-2 region. From the outputs, I can see:

- S3 Bucket Name: 000384376621-us-west-2-big-data-agent-workshop
- IAM Role ARN: arn:aws:iam::000384376621:role/AwsLabBigDataAgentStack-BigDataAgentRole-2oCLXxnlPxXX

Now, let's check if there are files in the data/ prefix of this bucket:
Tool #3: use_aws


...

The crawler is still running. Let's wait for another 15 seconds and check again:
Tool #11: sleep
Let's check the status again:
Tool #12: use_aws


╭─── 🚀 Check crawler status after waiting again ───╮
│                                                   │
│  Service:        glue                             │
│  Operation:      get_crawler                      │
│  Region:         us-west-2                        │
│  Parameters:                                      │
│    • Name:       AwsLabBigDataAgentStack-Crawler  │
│                                                   │
╰───────────────────────────────────────────────────╯

Great! The crawler has completed successfully and is now in the "READY" state.

### Step 6: Verify results

Let's check the tables that were created in the database:
Tool #13: use_aws


╭─ 🚀 List tables in the database after crawling ──╮
│                                                  │
│  Service:            glue                        │
│  Operation:          get_tables                  │
│  Region:             us-west-2                   │
│  Parameters:                                     │
│    • DatabaseName:   awslabbigdataagentstack_db  │
│                                                  │
╰──────────────────────────────────────────────────╯

Now, let's prepare the final response in the requested JSON format:

```json
{
  "catalog_db_name": "awslabbigdataagentstack_db",
  "catalog_table_names": ["data"]
}
```

I've successfully completed all the requested tasks:

1. Found the location of the data in the S3 bucket "000384376621-us-west-2-big-data-agent-workshop" with files under the "data/" prefix.
2. Found an existing database "awslabbigdataagentstack_db" that had no tables.
3. Created a new Glue crawler named "AwsLabBigDataAgentStack-Crawler" targeting the S3 data path.
4. Started the crawler and monitored its status until it completed successfully.
5. Verified that the crawler created a table named "data" in the database.

The crawler has successfully created a table with partitioning based on taxi_class, year, and month, which matches the directory structure in the S3 bucket. The table contains data from both green and yellow taxi trip data for May and June 2025.Now, let's prepare the final response in the requested JSON

In [15]:
from utils_big_data import print_tokens_costs

# Let's print the token costs
print_tokens_costs(crawl_response)


{'latencyMs': 58838}
{'inputTokens': 81415, 'outputTokens': 2848, 'totalTokens': 84263}
Time to research = 58.838 seconds
Input Token Costs = $0.244245;
Output Token Costs = $0.042719999999999994
Total Token Costs = $0.28696499999999997


## Extract Glue Database and Tables Names
Parse the agent response to extract the Glue database name and Table names for use in subsequent queries.

In [16]:
# Strands Agenst allows you to extract information in a structured dictionary output.

# Define the structured template
class GlueDbTableInfo(BaseModel):
    catalog_db_name: str = Field(description="Name of the Glue Catalog Database")
    catalog_table_names: List[str] = Field(description="List of glue table names")

# Use the structured_output method
glue_db_table_info = crawl_agent.structured_output(GlueDbTableInfo, f"Extract Glue Catalog Database name and the list of glue table names")
print(glue_db_table_info)
catalog_db_name = glue_db_table_info.catalog_db_name
catalog_table_names = glue_db_table_info.catalog_table_names


Tool #14: GlueDbTableInfo
catalog_db_name='awslabbigdataagentstack_db' catalog_table_names=['data']


AWS Labs has created many MCP servers for AI Agent developers to consume. One of them is AWS data processing MCP server: https://awslabs.github.io/mcp/servers/aws-dataprocessing-mcp-server <br><br>
This MCP server exposes AI Agent tools to do operations on big data using AWS Glue, Amazon EMR, and Amazon Athena.<br><br>
The best part is that we don't have to know the intricacies of these services other than high level basics.<br><br>
Let's create an MCP client for this MCP server.

## MCP Client Setup
We wll get tools exposed by an MCP server to discover partition columns and keys from the data in S3.<br><br>
Initialize the AWS Data Processing MCP server client to provide AI agents with AWS Glue, EMR, and Athena capabilities.

In [17]:
# Import MCP client libraries
from mcp import stdio_client, StdioServerParameters
from strands.tools.mcp import MCPClient
import boto3
session = boto3.Session()
credentials = session.get_credentials()

# Create MCP client for AWS data processing server
# This provides tools for Glue, EMR, and Athena operations
data_mcp_client = MCPClient(lambda: stdio_client(
    StdioServerParameters(
        command="uvx",  # Use uvx to run the MCP server
        args= [
            "awslabs.aws-dataprocessing-mcp-server@latest",
            "--allow-write",  # Enable write operations
        ],
        env= {
            "AWS_ACCESS_KEY_ID": credentials.access_key,
            "AWS_SECRET_ACCESS_KEY": credentials.secret_key,
            "AWS_SESSION_TOKEN": credentials.token,
            "FASTMCP_LOG_LEVEL": "ERROR",  # Minimize logging noise
            "AWS_REGION": session.region_name      # Set AWS region
      }
    )
))


In [18]:
import boto3

sts_client = boto3.client('sts')
response = sts_client.get_caller_identity()

arn = response['Arn']
# Example: arn:aws:sts::123456789012:assumed-role/MyRole/MySession
if "assumed-role" in arn:
    role_name = arn.split('/')[-2]
    print(f"IAM Role Name: {role_name}")
else:
    print("Not an assumed role session.")

IAM Role Name: AmazonSageMaker-ExecutionRole-20250205T203491


## Print the tools offered by the MCP Server
This will help you understand what the tools do. This is extracted by looking at doc strings or tool spec of the tools.<br><br>
Agents send the doc string or the tool spec of the tools to the LLM along with a task.<br>
This helps LLM to reason and decide which tool to use for which task.

In [19]:
# Connect to MCP client and execute task
import json

tools_char_count = 0
tool_count = 0
with data_mcp_client:
    # Get available tools from the MCP server
    data_tools = data_mcp_client.list_tools_sync()
    # first_tool = data_tools[0]
    # print(dir(first_tool))    
    # Iterate through each tool
    for tool in data_tools:
        print(f"Tool: {tool.tool_name}")
        if hasattr(tool, 'tool_spec'):            
            print(f"Tool: {tool.tool_spec['description']}")
            # print(json.dumps(tool.tool_spec, indent=2)) # uncomment this to see function parameters and what they mean   
            tools_char_count += len(json.dumps(tool.tool_spec))
            tool_count += 1
        print("-" * 50)
print(f"The number of characters in the spec of all the {tool_count} tools = {tools_char_count}")

Tool: manage_aws_glue_databases
Tool: Manage AWS Glue Data Catalog databases with both read and write operations.

        This tool provides operations for managing Glue Data Catalog databases, including creating,
        updating, retrieving, listing, and deleting databases. It serves as the primary mechanism
        for database management within the AWS Glue Data Catalog.

        ## Requirements
        - The server must be run with the `--allow-write` flag for create-database, update-database, and delete-database operations
        - Appropriate AWS permissions for Glue Data Catalog operations

        ## Operations
        - **create-database**: Create a new database in the Glue Data Catalog
        - **delete-database**: Delete an existing database from the Glue Data Catalog
        - **get-database**: Retrieve detailed information about a specific database
        - **list-databases**: List all databases in the Glue Data Catalog
        - **update-database**: Update an existin

### Understand the content that the tables have and what the columns mean. 
Store this in a file and pass it to the agent. This will help agent construct SQL queries properly.


In [20]:
# Get the definition of the table and columns and store it in a file.
table_def_system_prompt = f"""
You are an expert AWS data analyst assistant specializing in querying data stored in S3 data lakes using AWS Glue and Amazon Athena.

## IMPORTANT: 
- For any long running jobs, sleep for 20 seconds and check status until the job is finished. Do not give up until the job actually finishes.
"""
query_in = f"""Query the database {catalog_db_name}, identify the tables in it, identify the columns and their types. Gather more business context of what the database, tables, and the columns are storing by sampling a few rows of data. Then store the database name, table names, their purpose; and column name and its type and purpose in a json file named metadata.json."""

column_def_response = ""
# Get the data processing tools from MCP server
with data_mcp_client:
    data_tools = data_mcp_client.list_tools_sync()
    curated_data_tools = ['manage_aws_athena_query_executions', 'manage_aws_glue_tables']

    # Extract just the tools that we need.
    filtered_tools = [tool for tool in data_tools if tool.tool_name in curated_data_tools]
    filtered_tools += [file_write, file_read, sleep]

    # Pass the system prompt, the LLM we use with bedrock, and all the tools to the agent
    column_def = Agent(system_prompt = table_def_system_prompt, model=model, tools=filtered_tools)
    # Invoke the agent with each of the query
    column_def_response = column_def(query_in)

I'll help you query the database, identify tables, analyze their structure, and gather business context by sampling data. Let's break this down into steps:

1. First, I'll list the tables in the database "awslabbigdataagentstack_db"
2. For each table, I'll get the schema details (columns and types)
3. Sample data from each table to understand the business context
4. Compile all this information into a metadata.json file

Let's start by listing the tables in the database:
Tool #1: manage_aws_glue_tables
Now that I've identified there's one table named "data" in the database, let's get more detailed information about this table:
Tool #2: manage_aws_glue_tables
Now, let's sample some data from this table to understand its business context better:
Tool #3: manage_aws_athena_query_executions
I need to specify an output location for the query results. Let's try again with a result configuration:
Tool #4: manage_aws_athena_query_executions
Let's wait for the query to complete and then check t

╔═ File Write Operation ═╗
║                        ║
║ Path: metadata.json    ║
║ Size: 5344 characters  ║
║                        ║
╚════════════════════════╝

╔═════════════ Write Successful ═════════════╗
║ File written successfully to metadata.json ║
╚════════════════════════════════════════════╝

I've successfully analyzed the database and created a comprehensive metadata.json file. Here's a summary of what I found:

1. **Database**: `awslabbigdataagentstack_db`
   - Purpose: Stores NYC taxi trip data for analysis and reporting

2. **Table**: `data`
   - Contains detailed NYC taxi trip records
   - Stored in Parquet format in S3
   - Contains over 9 million records
   - Partitioned by taxi_class (yellow/green), year, and month

3. **Data Content**:
   - The data represents NYC taxi trips with detailed information about:
     - Pickup and dropoff times and locations
     - Trip distances and durations
     - Fare details (base fare, tips, taxes, surcharges)
     - Payment methods
     - Passenger counts
     - Taxi types (yellow or green)

4. **Key Insights**:
   - Data includes both yellow and green taxi trips
   - Currently contains data for May and June 2025
   - Different datetime fields are used for different taxi types (lpep_* for green taxis, tpep_* for yellow taxis)
   -